<a href="https://colab.research.google.com/github/Shivxnshjasathi/Chess.ai/blob/main/chessai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install chess

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 7.5 MB/s eta 0:00:00


In [4]:
import chess
import random

class ChessAI:
    def __init__(self, depth=3):
        self.depth = depth
        self.move_scores = {}
        self.move_history = []
        self.exploration_rate = 0.1

        # Piece-Square Tables (Positional score based on the location of each piece)
        self.piece_square_tables = {
            chess.PAWN: [
                0, 0, 0, 0, 0, 0, 0, 0,
                50, 50, 50, 50, 50, 50, 50, 50,
                10, 10, 20, 30, 30, 20, 10, 10,
                5, 5, 10, 25, 25, 10, 5, 5,
                0, 0, 0, 20, 20, 0, 0, 0,
                5, -5, -10, 0, 0, -10, -5, 5,
                5, 10, 10, -20, -20, 10, 10, 5,
                0, 0, 0, 0, 0, 0, 0, 0
            ],
            chess.KNIGHT: [
                -50, -40, -30, -30, -30, -30, -40, -50,
                -40, -20, 0, 5, 5, 0, -20, -40,
                -30, 5, 10, 15, 15, 10, 5, -30,
                -30, 0, 15, 20, 20, 15, 0, -30,
                -30, 5, 15, 20, 20, 15, 5, -30,
                -30, 0, 10, 15, 15, 10, 0, -30,
                -40, -20, 0, 0, 0, 0, -20, -40,
                -50, -40, -30, -30, -30, -30, -40, -50
            ],
            chess.BISHOP: [
                -20, -10, -10, -10, -10, -10, -10, -20,
                -10, 5, 0, 0, 0, 0, 5, -10,
                -10, 10, 10, 10, 10, 10, 10, -10,
                -10, 0, 10, 10, 10, 10, 0, -10,
                -10, 5, 5, 10, 10, 5, 5, -10,
                -10, 0, 5, 10, 10, 5, 0, -10,
                -10, 0, 0, 0, 0, 0, 0, -10,
                -20, -10, -10, -10, -10, -10, -10, -20
            ],
            chess.ROOK: [
                0, 0, 0, 0, 0, 0, 0, 0,
                5, 10, 10, 10, 10, 10, 10, 5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                -5, 0, 0, 0, 0, 0, 0, -5,
                0, 0, 0, 5, 5, 0, 0, 0
            ],
            chess.QUEEN: [
                -20, -10, -10, -5, -5, -10, -10, -20,
                -10, 0, 0, 0, 0, 0, 0, -10,
                -10, 0, 5, 5, 5, 5, 0, -10,
                -5, 0, 5, 5, 5, 5, 0, -5,
                0, 0, 5, 5, 5, 5, 0, -5,
                -10, 5, 5, 5, 5, 5, 0, -10,
                -10, 0, 5, 0, 0, 0, 0, -10,
                -20, -10, -10, -5, -5, -10, -10, -20
            ],
            chess.KING: [
                -30, -40, -40, -50, -50, -40, -40, -30,
                -30, -40, -40, -50, -50, -40, -40, -30,
                -30, -40, -40, -50, -50, -40, -40, -30,
                -30, -40, -40, -50, -50, -40, -40, -30,
                -20, -30, -30, -40, -40, -30, -30, -20,
                -10, -20, -20, -20, -20, -20, -20, -10,
                20, 20, 0, 0, 0, 0, 20, 20,
                20, 30, 10, 0, 0, 10, 30, 20
            ]
        }

    def choose_move(self, board):
        legal_moves = list(board.legal_moves)
        best_move = None
        best_score = float('-inf')
        alpha = float('-inf')
        beta = float('inf')

        legal_moves = self.order_moves(board, legal_moves)

        for move in legal_moves:
            board.push(move)
            score = -self.quiescence_search(board, self.depth - 1, -beta, -alpha, not board.turn)
            board.pop()

            if score > best_score:
                best_score = score
                best_move = move

            alpha = max(alpha, score)
            if alpha >= beta:
                break

        self.update_move_scores(board, best_move, best_score)
        self.update_move_history(best_move)

        return best_move

    def minimax(self, board, depth, alpha, beta, maximizing_player):
        if depth == 0 or board.is_game_over():
            return self.evaluate_board(board)

        if maximizing_player:
            max_eval = float('-inf')
            for move in board.legal_moves:
                board.push(move)
                eval = self.minimax(board, depth - 1, alpha, beta, False)
                board.pop()
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for move in board.legal_moves:
                board.push(move)
                eval = self.minimax(board, depth - 1, alpha, beta, True)
                board.pop()
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def quiescence_search(self, board, depth, alpha, beta, maximizing_player):
        if depth == 0:
            return self.evaluate_board(board)

        if maximizing_player:
            max_eval = float('-inf')
            for move in board.legal_moves:
                if board.is_capture(move):
                    board.push(move)
                    eval = -self.quiescence_search(board, depth - 1, -beta, -alpha, False)
                    board.pop()
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)
                    if beta <= alpha:
                        break
            return max_eval
        else:
            min_eval = float('inf')
            for move in board.legal_moves:
                if board.is_capture(move):
                    board.push(move)
                    eval = -self.quiescence_search(board, depth - 1, -beta, -alpha, True)
                    board.pop()
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)
                    if beta <= alpha:
                        break
            return min_eval

    def evaluate_board(self, board):
        piece_values = {
            chess.PAWN: 10,
            chess.KNIGHT: 30,
            chess.BISHOP: 30,
            chess.ROOK: 50,
            chess.QUEEN: 90,
            chess.KING: 0
        }

        score = 0
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                value = piece_values[piece.piece_type]
                if piece.color == chess.WHITE:
                    score += value
                    score += self.piece_square_tables.get(piece.piece_type, [0]*64)[square]
                else:
                    score -= value
                    score -= self.piece_square_tables.get(piece.piece_type, [0]*64)[chess.square_mirror(square)]

        return score

    def order_moves(self, board, moves):
        def move_score(move):
            score = self.move_scores.get(move.uci(), 0)
            if random.random() < self.exploration_rate:
                score += random.uniform(-200, 200)
            return score
        return sorted(moves, key=lambda move: move_score(move), reverse=True)

    def update_move_scores(self, board, move, score):
        move_uci = move.uci()
        if move_uci not in self.move_scores:
            self.move_scores[move_uci] = score
        else:
            self.move_scores[move_uci] = (self.move_scores[move_uci] + score) / 2

    def update_move_history(self, move):
        self.move_history.append(move.uci())
        if len(self.move_history) > 5:
            self.move_history.pop(0)

def print_board(board):
    piece_map = {
        chess.PAWN: 'P', chess.KNIGHT: 'N', chess.BISHOP: 'B',
        chess.ROOK: 'R', chess.QUEEN: 'Q', chess.KING: 'K'
    }

    piece_map.update({
        chess.PAWN: 'p', chess.KNIGHT: 'n', chess.BISHOP: 'b',
        chess.ROOK: 'r', chess.QUEEN: 'q', chess.KING: 'k'
    })

    lines = []
    for rank in range(7, -1, -1):
        row = [f"{rank + 1} "]
        for file in range(8):
            square = chess.square(file, rank)
            piece = board.piece_at(square)
            if piece:
                row.append(piece_map[piece.piece_type].upper() if piece.color == chess.WHITE else piece_map[piece.piece_type])
            else:
                row.append('.')
        lines.append(' '.join(row))

    lines.append('   a b c d e f g h')
    print('\n'.join(lines))

def play_game():
    board = chess.Board()
    ai = ChessAI(depth=3)

    human_score = 0
    ai_score = 0

    while not board.is_game_over():
        if board.turn == chess.WHITE:
            print_board(board)
            move = input("Enter your move (e.g., e2e4) or 'quit' to exit: ")

            if move.lower() == "quit":
                print("You quit the game.")
                break

            try:
                board.push_san(move)
            except ValueError:
                print("Invalid move. Try again.")
                continue
        else:
            move = ai.choose_move(board)
            board.push(move)
            print(f"AI moved: {move}")

        current_score = ai.evaluate_board(board)
        if board.turn == chess.WHITE:
            human_score = current_score
        else:
            ai_score = current_score

        print(f"Current Score -> Human: {human_score} | AI: {ai_score}")

    if board.is_game_over():
        print_board(board)
        print("Game Over")
        print(f"Result: {board.result()}")

if __name__ == "__main__":
    play_game()

8  r n b q k b n r
7  p p p p p p p p
6  . . . . . . . .
5  . . . . . . . .
4  . . . . . . . .
3  . . . . . . . .
2  P P P P P P P P
1  R N B Q K B N R
   a b c d e f g h
Enter your move (e.g., e2e4) or 'quit' to exit: e2e4
Current Score -> Human: 0 | AI: -25
AI moved: h7h5
Current Score -> Human: 20 | AI: -25
8  r n b q k b n r
7  p p p p p p p .
6  . . . . . . . .
5  . . . . . . . p
4  . . . . P . . .
3  . . . . . . . .
2  P P P P . P P P
1  R N B Q K B N R
   a b c d e f g h
Enter your move (e.g., e2e4) or 'quit' to exit: g1f3
Current Score -> Human: 20 | AI: 70
AI moved: h5h4
Current Score -> Human: 75 | AI: 70
8  r n b q k b n r
7  p p p p p p p .
6  . . . . . . . .
5  . . . . . . . .
4  . . . . P . . p
3  . . . . . N . .
2  P P P P . P P P
1  R N B Q K B . R
   a b c d e f g h
Enter your move (e.g., e2e4) or 'quit' to exit: b1c3
Current Score -> Human: 75 | AI: 125
AI moved: g8f6
Current Score -> Human: 75 | AI: 125
8  r n b q k b . r
7  p p p p p p p .
6  . . . . . n . .
5  . . 